In [5]:
pip install selenium

     ---------------------------------------- 6.3/6.3 MB 2.5 MB/s eta 0:00:00
     -------------------------------------- 384.9/384.9 kB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 58.3/58.3 kB 3.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [29]:
#import libraires
import csv
from bs4 import BeautifulSoup

In [30]:
#Fire fox and chrome
from selenium import webdriver

### startup the web driver

In [31]:
driver = webdriver.Chrome()

In [32]:
url = 'https://www.amazon.com'
driver.get(url)

In [5]:
#بتخليني اسرش جوه ال url النا بحطه الهو الtemplate
def get_url(search_text):
    """Generate a url from search text"""
    template = 'https://www.amazon.com/s?k={}&crid=2JTVXJWSKO5WZ&sprefix=ultrawide+monitor%2Caps%2C245&ref=nb_sb_noss_1'
    search_term = search_text.replace(' ','+')
    return template.format(search_term)

In [6]:
url = get_url('ultrawide monitor')
print(url)

https://www.amazon.com/s?k=ultrawide+monitor&crid=2JTVXJWSKO5WZ&sprefix=ultrawide+monitor%2Caps%2C245&ref=nb_sb_noss_1


In [7]:
driver.get(url)

### extract the collection 

In [8]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [9]:
results = soup.find_all("div", {"data-component-type" : "s-search-result"})

In [10]:
len(results)

22

### prototype the record

In [23]:
item = results[0] #first product in amazon 

In [25]:
atag = item.h2.a #the header of the first product in amazon 

In [65]:
#the header with html tags
# atag 

In [32]:
desc = atag.text.strip() # we used text.strip() to remove unseful html tag and keep the header only 

'Deco Gear 35” Curved Gaming Ultrawide Monitor, 3440x1440, 120hz, 1ms MPRT, 21:9, 99% sRGB'

### fetch price from amazon 

In [51]:
price = soup.find_all("span", {"class": "a-price"})

In [64]:
price[0].text

'$349.99$349.99'

### fetch rating from amazon

In [55]:
rating = soup.find_all("i", {"class": "a-icon a-icon-star-small a-star-small-4-5 aok-align-bottom"})

In [57]:
rating[0].text

'4.6 out of 5 stars'

### fetch review count for every product in amazon

In [67]:
review_count = soup.find_all("span", {"class": "a-size-base s-underline-text"})

In [70]:
review_count[0].text

'(733)'

## Generalize the pattern 

In [11]:
def extract_record(item):
    """Extract and return data from a single record"""
    
    # description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    # product price
    price = soup.find("span", {"class": "a-price"}).text
    
    # rating and review count
    rating = item.i.text
    review_count = soup.find("span", {"class": "a-size-base s-underline-text"}).text
    
    result = (description, price, rating, review_count, url)
    
    return result

In [12]:
records = []

# get all search results
results = soup.find_all('div', {'data-component-type': 's-search-result'})

# extract data from each results
for item in results:
    records.append(extract_record(item))

### Handling errors

In [13]:
def extract_record(item):
    """Extract and return data from a single record"""
    
    # description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    try:
        # product price
        price = soup.find("span", {"class": "a-price"}).text
    except AttributeError:
        return
    
    try:
        # rating and review count
        rating = item.i.text
        review_count = soup.find("span", {"class": "a-size-base s-underline-text"}).text
    except AttributeError:
        rating = '' #empty string 
        review_count = ''# empty string 
        
    result = (description, price, rating, review_count, url) #tuple
    
    return result

In [14]:
records = []

# get all search results
results = soup.find_all('div', {'data-component-type': 's-search-result'})

# extract data from each results
for item in results:
    records.append(extract_record(item))

### Getting to the next page

In [ ]:
def get_url(search_text):
    """Generate a url from search text"""
    template = 'https://www.amazon.com/s?k={}&ref=nb_sb_noss_1'
    search_term = search_text.replace(' ', '+')
    
    # add term query to url
    url = template.format(search_term)
    
    # add page query placeholder
    url += '&page={}'
        
    return url

### putting it all together

In [27]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver


def get_url(search_text):
    """Generate a url from search text"""
    template = 'https://www.amazon.com/s?k={}&ref=nb_sb_noss_1'
    search_term = search_text.replace(' ', '+')
    
    # add term query to url
    url = template.format(search_term)
    
    # add page query placeholder
    url += '&page{}'
        
    return url

def extract_record(item):
    """Extract and return data from a single record"""
    
    # description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    try:
        # product price
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    try:
        # rating and review count
        rating = soup.find("i", {"class": "a-icon a-icon-star-small a-star-small-4-5 aok-align-bottom"}).text
        review_count = soup.find("span", {"class": "a-size-base s-underline-text"}).text
    except AttributeError:
        rating = ''
        review_count = ''
        
    result = (description, price, rating, review_count, url)
    
    return result

def main(search_term):
    """Run main program routine"""
    
    # startup the webdriver
    driver = webdriver.Chrome()
    records = []
    url = get_url(search_term)
    
    for page in range(1, 21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all("div", {"data-component-type" : "s-search-result"})
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
                
    driver.close()
    
    # save data to csv file (excel sheet)
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'ReviewCount', 'Url'])
        writer.writerows(records)

In [28]:
main('ultrawide monitor')

In [34]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
results = soup.find_all('div', {'data-component-type': 's-search-result'})
item = results[0] #prototype model

In [57]:
item.i.text

'4.6 out of 5 stars'

In [67]:
review_count = item.find("span", {"class": "a-size-base s-underline-text"}).text
review_count

'(733)'

In [73]:
#import the used libraries
import csv 
from bs4 import BeautifulSoup
from selenium import webdriver

def get_url(search_text):
    """take the search term and generate it in all pages in the web"""
    template = "https://www.amazon.com/s?k={}&crid=15DNYWRD1AUTF&sprefix=ultrawide+monitor%2Caps%2C208&ref=nb_sb_noss_1"
    search_term = search_text.replace(' ','+') #replace empty string to +
    url = template.format(search_term) #then put it in the template url variable
    url += '&page{}' #add page query placeholder
    return url

def extract_record(item):
    """extract and return the data from single records"""
    #description and url 
    atag = item.h2.a #html header
    desc  = atag.text.strip() #header without html tag
    url = 'https://www.amazon.com' + atag.get('href')
    
    #fetch the price
    try:
        price_parent = item.find("span", {"class":"a-price"})
        price = price_parent.find("span", {"class": "a-offscreen"}).text
    except AttributeError: 
        return
    
    #fetch the review and count review 
    try:
        review = item.i.text #product rate
        review_count = item.find("span", {"class": "a-size-base s-underline-text"}).text
    except AttributeError:
        review = ''
        review_count = ''
        
    data = (desc, price, review, review_count, url) #all data
    return data


def main(search_term):
    #chrome driver
    driver = webdriver.Chrome()
    records = []
    url = get_url(search_term)
    
    for page in range(1,21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
    driver.close()
    
    # save data to csv file (excel sheet)
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'ReviewCount', 'Url'])
        writer.writerows(records)

In [74]:
main("ultrawide monitor")